In [29]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irfan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:
# Define stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Sample documents
documents = [
    "Rafael Nadal Joins Roger Federer in Missing U.S. Open",
    "Rafael Nadal Is Out of the Australian Open",
    "Biden Announces Virus Measures",
    "Biden's Virus Plans Meet Reality",
    "Where Biden's Virus Plan Stands"
]

df = pd.DataFrame(documents, columns=["Document"])

pd.set_option('display.max_colwidth', None)
print(df)

                                                Document
0  Rafael Nadal Joins Roger Federer in Missing U.S. Open
1             Rafael Nadal Is Out of the Australian Open
2                         Biden Announces Virus Measures
3                       Biden's Virus Plans Meet Reality
4                        Where Biden's Virus Plan Stands


In [33]:
# Preprocess function
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Remove non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize tokens
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]

# Print the preprocessed documents in the desired format
print(preprocessed_documents)

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'], ['rafael', 'nadal', 'australian', 'open'], ['biden', 'announces', 'virus', 'measure'], ['biden', 'virus', 'plan', 'meet', 'reality'], ['biden', 'virus', 'plan', 'stand']]


In [35]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)

# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [37]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

In [39]:
# 6. Interpret Results

# empty list to store dominant topic labels for each document
article_labels = []

# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to bag-of-words representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # append the dominant topic to the list
    article_labels.append(dominant_topic)

# Create DataFrame
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                                 Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U.S. Open      0
1             Rafael Nadal Is Out of the Australian Open      0
2                         Biden Announces Virus Measures      1
3                       Biden's Virus Plans Meet Reality      1
4                        Where Biden's Virus Plan Stands      1



In [41]:
# Print the top terms for each topic
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
print()

Top Terms for Each Topic:
Topic 0:
- "nadal" (weight: 0.131)
- "rafael" (weight: 0.131)
- "open" (weight: 0.131)
- "federer" (weight: 0.079)
- "roger" (weight: 0.079)
- "missing" (weight: 0.079)
- "join" (weight: 0.079)
- "australian" (weight: 0.079)
- "biden" (weight: 0.027)
- "virus" (weight: 0.027)
Topic 1:
- "virus" (weight: 0.166)
- "biden" (weight: 0.166)
- "plan" (weight: 0.119)
- "measure" (weight: 0.071)
- "announces" (weight: 0.071)
- "meet" (weight: 0.071)
- "stand" (weight: 0.071)
- "reality" (weight: 0.071)
- "open" (weight: 0.024)
- "rafael" (weight: 0.024)

